# Data Preprocessing Steps

1. Running FFT analysis on SEEG data - log seizure times using Z
2. Running FFT projection into 2D tensors, or 3D tensors - log seizure times using preivous step
2D = [numsamps, numfreqs, W, H]
3D = [numsamps, numfreqs, W, H, D]

To project onto 2D or 3D, we will define a mesh grid of the image in brain MRI space. So the electrodes need to have defined xyz coordinates in the MRI (e.g. say freesurfer, or flirt space) when doing reconstructions.

Then each step will be saved with the corresponding data samples as a .npz file.